In [2]:
# ==================================================
# 1. IMPORTAÇÕES
# ==================================================
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import (
    accuracy_score, roc_auc_score, recall_score,
    precision_score, f1_score, cohen_kappa_score,
    matthews_corrcoef, confusion_matrix
)
import joblib

# Para reproduzir os mesmos resultados
RANDOM_STATE = 123

# ==================================================
# 2. CARREGAR DADOS
# ==================================================
data = pd.read_csv('HR-Employee-Attrition.csv')

print("Distribuição da variável Attrition:")
print(data['Attrition'].value_counts())
print("\n" + "="*50 + "\n")

# ==================================================
# 3. SEPARAR FEATURES E TARGET
# ==================================================
X = data.drop('Attrition', axis=1)
y = data['Attrition'].map({'Yes': 1, 'No': 0})  # converte para 0/1

# ==================================================
# 4. DIVISÃO TREINO/TESTE (70/30) – igual ao padrão do PyCaret
# ==================================================
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, stratify=y, random_state=RANDOM_STATE
)

print(f"Tamanho do treino: {X_train.shape}")
print(f"Tamanho do teste: {X_test.shape}")
print("="*50 + "\n")

# ==================================================
# 5. PRÉ‑PROCESSAMENTO (IDENTIFICAR COLUNAS CATEGÓRICAS)
# ==================================================
# Vamos inspecionar os tipos. No dataset IBM as categóricas são:
categorical_cols = X.select_dtypes(include=['object']).columns.tolist()
# Além disso, algumas variáveis que parecem numéricas podem ser categóricas
# (ex: Education, EnvironmentSatisfaction etc.) – mas o PyCaret as trata como numéricas.
# Vou manter todas as numéricas como estão.

print("Colunas categóricas detectadas:", categorical_cols)
print("="*50 + "\n")

# Criamos um transformer que aplica OneHotEncoder nas categóricas
# e deixa as numéricas inalteradas (passthrough)
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), categorical_cols)
    ],
    remainder='passthrough'  # as numéricas passam direto
)

# ==================================================
# 6. PIPELINE COMPLETO (PRÉ‑PROC + MODELO)
# ==================================================
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', GaussianNB())
])

# ==================================================
# 7. VALIDAÇÃO CRUZADA INICIAL (20 FOLDS) – opcional, só para ver as métricas
# ==================================================
print("Executando validação cruzada com 20 folds (modelo padrão)...")
cv = StratifiedKFold(n_splits=20, shuffle=True, random_state=RANDOM_STATE)

# Função para calcular várias métricas
def compute_metrics(y_true, y_pred, y_proba=None):
    metrics = {
        'Accuracy': accuracy_score(y_true, y_pred),
        'Recall': recall_score(y_true, y_pred, zero_division=0),
        'Precision': precision_score(y_true, y_pred, zero_division=0),
        'F1': f1_score(y_true, y_pred, zero_division=0),
        'Kappa': cohen_kappa_score(y_true, y_pred),
        'MCC': matthews_corrcoef(y_true, y_pred)
    }
    if y_proba is not None:
        metrics['AUC'] = roc_auc_score(y_true, y_proba[:, 1])
    return metrics

# Coletar resultados de cada fold
cv_results = []
for train_idx, val_idx in cv.split(X_train, y_train):
    X_tr, X_val = X_train.iloc[train_idx], X_train.iloc[val_idx]
    y_tr, y_val = y_train.iloc[train_idx], y_train.iloc[val_idx]
    
    pipeline.fit(X_tr, y_tr)
    y_pred = pipeline.predict(X_val)
    y_proba = pipeline.predict_proba(X_val)
    
    cv_results.append(compute_metrics(y_val, y_pred, y_proba))

# Mostrar média e desvio padrão (igual à tabela do PyCaret)
cv_df = pd.DataFrame(cv_results)
print("\nMédias da validação cruzada (20 folds):")
print(cv_df.mean().round(4))
print("\nDesvios padrão:")
print(cv_df.std().round(4))
print("="*50 + "\n")

# ==================================================
# 8. OTIMIZAÇÃO DE HIPERPARÂMETROS (FOCO EM RECALL)
# ==================================================
print("Otimizando hiperparâmetros (var_smoothing) para maximizar recall...")

# Grid de var_smoothing (valores comuns para GaussianNB)
param_grid = {
    'classifier__var_smoothing': np.logspace(-12, 0, 13)  # de 1e-12 até 1
}

grid_search = GridSearchCV(
    pipeline,
    param_grid,
    scoring='recall',
    cv=StratifiedKFold(n_splits=20, shuffle=True, random_state=RANDOM_STATE),
    n_jobs=-1,
    verbose=1
)

grid_search.fit(X_train, y_train)

print(f"Melhor var_smoothing: {grid_search.best_params_}")
print(f"Melhor recall médio na CV: {grid_search.best_score_:.4f}")
print("="*50 + "\n")

# ==================================================
# 9. AVALIAÇÃO NO CONJUNTO DE TESTE
# ==================================================
best_model = grid_search.best_estimator_

y_test_pred = best_model.predict(X_test)
y_test_proba = best_model.predict_proba(X_test)

test_metrics = compute_metrics(y_test, y_test_pred, y_test_proba)

print("Métricas no conjunto de teste:")
test_df = pd.DataFrame([test_metrics])
test_df.insert(0, 'Model', 'Naive Bayes (tunado)')
print(test_df.round(4).to_string(index=False))
print("="*50 + "\n")

# Opcional: exibir as primeiras previsões (como no PyCaret)
X_test_with_pred = X_test.copy()
X_test_with_pred['Attrition'] = y_test.map({1: 'Yes', 0: 'No'})
X_test_with_pred['Label'] = ['Yes' if p == 1 else 'No' for p in y_test_pred]
X_test_with_pred['Score'] = y_test_proba[:, 1]  # probabilidade da classe positiva

print("Primeiras previsões (teste):")
print(X_test_with_pred[['Age', 'DailyRate', 'DistanceFromHome', 'EmployeeNumber',
                        'MonthlyIncome', 'Attrition', 'Label', 'Score']].head())
print("="*50 + "\n")

# ==================================================
# 10. FINALIZAR MODELO (TREINAR COM TODOS OS DADOS)
# ==================================================
print("Finalizando modelo (treinando com todos os dados)...")
final_model = grid_search.best_estimator_
final_model.fit(X, y)  # reajusta com o dataset completo
print("Modelo finalizado!")
print("="*50 + "\n")

# ==================================================
# 11. SALVAR MODELO PARA PRODUÇÃO
# ==================================================
model_filename = 'modelo_naive_bayes_02_02_2026.pkl'
joblib.dump(final_model, model_filename)
print(f"Modelo salvo como '{model_filename}'")
print("Processo concluído com sucesso!")

Distribuição da variável Attrition:
Attrition
No     1233
Yes     237
Name: count, dtype: int64


Tamanho do treino: (1029, 34)
Tamanho do teste: (441, 34)

Colunas categóricas detectadas: ['BusinessTravel', 'Department', 'EducationField', 'Gender', 'JobRole', 'MaritalStatus', 'Over18', 'OverTime']

Executando validação cruzada com 20 folds (modelo padrão)...

Médias da validação cruzada (20 folds):
Accuracy     0.7163
Recall       0.7417
Precision    0.3358
F1           0.4595
Kappa        0.3052
MCC          0.3502
AUC          0.7748
dtype: float64

Desvios padrão:
Accuracy     0.0610
Recall       0.1384
Precision    0.0672
F1           0.0831
Kappa        0.1088
MCC          0.1201
AUC          0.0898
dtype: float64

Otimizando hiperparâmetros (var_smoothing) para maximizar recall...
Fitting 20 folds for each of 13 candidates, totalling 260 fits
Melhor var_smoothing: {'classifier__var_smoothing': 1e-12}
Melhor recall médio na CV: 0.7771

Métricas no conjunto de teste:
             